In [1]:
%pip install openai
from pprint import pprint
import openai
from openai import OpenAI
import os

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
No

In [4]:
openai.api_key = os.environ.get('OPENAI_API_KEY')


In [6]:
client = OpenAI(api_key=openai.api_key)    

In [10]:
#read in credit_risk_dataset.csv and chunk it into 1000 row chunks
import pandas as pd
df = pd.read_csv('credit_risk_dataset.csv')
#chunk into 1000 row chunks
df_chunks = [df[i:i+1000] for i in range(0,df.shape[0],1000)]
# write out chucks to separate json files
for i in range(len(df_chunks)):
    df_chunks[i].to_json(f'credit_risk_dataset_{i}.json', orient='records')

In [18]:
file = client.files.create(file=open('credit_risk_dataset_0.json', 'rb'), purpose='assistants')
file_id = file.id

In [19]:
print(file.id)
file_list = client.files.list()
print(file_list)

file-TVWLkt1zUO1RmiTw5QT9DvOp
SyncPage[FileObject](data=[FileObject(id='file-TVWLkt1zUO1RmiTw5QT9DvOp', bytes=285278, created_at=1699723808, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-vhJ701wWZmK9kf5RvbDec5GJ', bytes=285278, created_at=1699723792, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-qg7CwllzF0TFoS0LiSfmy7NZ', bytes=285278, created_at=1699723750, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-Yi8O05x6kCMPdDkI6KkaWgoA', bytes=285278, created_at=1699723190, filename='credit_risk_dataset_0.json', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-E1Rxe4e5I7RCklumeZVd0uCK', bytes=1804682, created_at=1699722932, filename='credit_risk_dataset.csv', object='file',

In [21]:
assistant = client.beta.assistants.create(
    instructions="This is a test assistant",
    model = "gpt-4-1106-preview",
    tools = [{"type":"retrieval"}],
    file_ids = [file_id]
)
print(assistant.id)

asst_72gLFIRPfYdZ8xxBJG6S21hI


In [22]:
# update assistant if you need to change the assistant
assistant = client.beta.assistants.update(
    assistant.id,
    instructions="This is a test assistant",
    model = "gpt-4-1106-preview",
    tools = [{"type":"retrieval"}],
    file_ids = [file_id]
)

In [23]:
# list assistants
assistants = client.beta.assistants.list(order="desc")

In [24]:
# lets suppose you want to use and existing assistant
assistant_id = 'asst_72gLFIRPfYdZ8xxBJG6S21hI'
assistant = client.beta.assistants.retrieve(assistant_id)


In [29]:
# create a thread
thread = client.beta.threads.create()
print(thread.id)

thread_3x9saDH1hkuUklyOOcL0pExD


In [32]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content="Hello, I'm a message"
)

In [35]:
# run the assistant to get a response
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "I want to buy a house",
)
print(run.id)

run_N8kJ9jL8WZLSZBR0yl7iI7XF


In [36]:
# retrieve the response
status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)
print(status)

Run(id='run_N8kJ9jL8WZLSZBR0yl7iI7XF', assistant_id='asst_72gLFIRPfYdZ8xxBJG6S21hI', cancelled_at=None, completed_at=1699724614, created_at=1699724612, expires_at=None, failed_at=None, file_ids=['file-TVWLkt1zUO1RmiTw5QT9DvOp'], instructions='I want to buy a house', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699724612, status='completed', thread_id='thread_3x9saDH1hkuUklyOOcL0pExD', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [37]:
# list out run steps
steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)
print(steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_HwNOt7JloAUQX4vuLttkE5lx', assistant_id='asst_72gLFIRPfYdZ8xxBJG6S21hI', cancelled_at=None, completed_at=1699724614, created_at=1699724613, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_N8kJ9jL8WZLSZBR0yl7iI7XF', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_nBNC8Pab10A443rMM4f1k1nw'), type='message_creation'), thread_id='thread_3x9saDH1hkuUklyOOcL0pExD', type='message_creation', expires_at=None)], object='list', first_id='step_HwNOt7JloAUQX4vuLttkE5lx', last_id='step_HwNOt7JloAUQX4vuLttkE5lx', has_more=False)


In [ ]:
# CANCEL A RUN - Dont do this if you want to use the run
run = client.beta.threads.runs.cancel(
    thread_id = thread.id,
    run_id = run.id
)
print(run)

In [39]:
# loop to see if run is complete
import time
while True:
    start = time.time()
    status = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
    if status.status == 'complete':
        break
    time.sleep(10)
    if time.time() - start > 300:   # 5 minutes
        break
print(status)

KeyboardInterrupt: 

In [41]:
# get all the messages
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
for message in messages:
    print(message.content)

[MessageContentText(text=Text(annotations=[], value="Hello! It looks like you're interested in buying a house and have uploaded a file that might be related to this process. How can I assist you with it today? If you need information extracted or summarized from the file, please let me know what you're looking for specifically."), type='text')]
[MessageContentText(text=Text(annotations=[], value="Hello, I'm a message"), type='text')]


In [ ]:
# clean up the assistant
assistant = client.beta.assistants.delete(assistant.id)
print(assistant)
